In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta, timezone
import matplotlib.dates as mdates

In [ ]:
path = "/home/alberto/Docker/Trading/analysis/json/data.json"
f = open (path, "r")
data_dict = json.loads(f.read())
data = data_dict['data']
print('DATETIME CREATION: ', data_dict['datetime_creation'])

In [ ]:
keys_obj = list(data['BTCUSDT'][-1].keys())
#['_id','price', price_%_1d','price_%_6h','price_%_3h','price_%_1h',
#'vol_1m', 'buy_vol_1m', buy_trd_1m
#'vol_5m','vol_5m_std','buy_vol_5m','buy_vol_5m_std','buy_trd_5m','buy_trd_5m_std',
#'vol_15m','vol_15m_std','buy_vol_15m','buy_vol_15m_std','buy_trd_15m','buy_trd_15m_std',
#'vol_30m','vol_30m_std','buy_vol_30m','buy_vol_30m_std','buy_trd_30m','buy_trd_30m_std',
#'vol_60m','vol_60m_std','buy_vol_60m','buy_vol_60m_std','buy_trd_60m','buy_trd_60m_std',
#'vol_24h','vol_24h_std','buy_vol_24h','buy_vol_24h_std','buy_trd_24h','buy_trd_24h_std']

In [ ]:
def getTimeseries(data, coin, key, days=1):
    timeseries = [obj[key] for obj in data[coin][-int(days*24*60):]]
    
    return np.array(timeseries)
    
def getDateSeries(coin, days=1):
    
    timeseries = [obj['_id'] for obj in data[coin]][-int(days*24*60):]
    date_series = []
    for date in timeseries:
        date_series.append(np.datetime64(date))
    
    return np.array(date_series)
    
def normalizeData(timeseries):
    mean = np.mean(timeseries)
    std = np.std(timeseries)
    
    normalized_data = (timeseries - mean) / std
    return normalized_data

def post_prices(n, prices, buy_volumes, volumes=None, volume_filter_min=None, volume_filter_max=None):
    '''
    this function anticipates eliminates the first n elements of the list.
    This is to evaluate if the buy_volume affects the prices in the future
    
    '''
    
    
    prices = prices[n:]
    buy_volumes = buy_volumes[:-n]
    
    if volume_filter_min == None:
        print('Correlation list prices and buy: ', np.corrcoef(prices,buy_volumes)[0,1]) 
    else:
        subset_prices = []
        subset_buyvolume = []


        for volume, price, buy_volume in zip(volumes, prices, buy_volumes):
            if volume > volume_filter_min and volume < volume_filter_max:
                subset_prices.append(price)
                subset_buyvolume.append(buy_volume)
                
        print('length of sample: ',len(subset_prices))
        print('Correlation post prices and buy: ', np.corrcoef(subset_prices,subset_buyvolume)[0,1])

def search_focus_points(data, period, volume_limit, buy_volume_limit, referenceTime_volume, referenceTime_Buyvolume):
    '''
    This function looks for all those points in time for each coin that have interesting events.
    '''
    response = {}
    for coin in data:
        for obs in data[coin][-int(period*24*60):]:
            if obs[referenceTime_volume] >= buy_volume_limit and obs[referenceTime_Buyvolume] >= referenceTime_Buyvolume:
                if coin not in response:
                    response[coin] = []
                response[coin].append(obs)
    
    return response

    


       
        

In [ ]:
DAYS=1/6

INTERVAL = int(DAYS * 8)
coin = 'LINKUSDT'


volumes = getTimeseries(data, coin, 'vol_15m',days=DAYS)
volumes_norm = normalizeData(volumes)
prices = getTimeseries(data, coin, 'price',days=DAYS)
prices_norm = normalizeData(prices)
buy_volumes = getTimeseries(data, coin, 'buy_vol_15m',days=DAYS)
buy_volumes_norm = normalizeData(buy_volumes)
dateseries = getDateSeries(coin, days=DAYS)

In [ ]:
fig, ax = plt.subplots(figsize=(30,20))
ax.plot(dateseries, volumes_norm, label='Volumes')
ax.plot(dateseries, prices_norm, label='Prices')
ax.plot(dateseries, buy_volumes_norm, label='Buy Volumes')
ax.xaxis.set_major_locator(mdates.HourLocator(interval=INTERVAL))
#ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d: %H-%M'))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%d/%m - %H:%M'))

ax.tick_params(axis='x', labelsize=10)
ax.tick_params(axis='y', labelsize=15)
ax.set_xlabel('Date')
ax.set_ylabel('Value')
ax.set_title('Time series plot with days on x-axis')
ax.legend(fontsize=30)
plt.show()

In [ ]:
#print('Correlation list prices and buy: ', np.corrcoef(prices,buy_volumes)[0,1])
#print('Correlation subset prices and buy: ', np.corrcoef(subset_prices,subset_buyvolume)[0,1])
post_prices(1, prices, buy_volumes, volumes, 1.1,3)